In [2]:
import boto3
import zipfile
import json
import io
import spacy
import re
import csv
import pandas as pd
import spacy_fastlang


In [3]:
# text_relevant_df = pd.read_csv("/Users/rlzhang1310/Coding/buntain/sentiment_data/text_relevant_df.csv")
text_relevant_df = pd.read_csv("/Users/rlzhang1310/Coding/buntain/sentiment_data/text_relevant_df.csv", encoding='utf-8', quoting=csv.QUOTE_ALL)
text_relevant_df.drop(['Unnamed: 0'], axis=1,inplace=True)
text_relevant_df


,tweet_id,project_substring,tweet_text,tweet_language,project_country,CampaignID
0,1387933360429748225,muda,Munarman telah diciduk ini lah dukungan dri an...,in,MLS,TMRC14_APAC_1
1,1366595382596149248,baru,RT @radenrauf: Apa yang bikin kamu gampang ilf...,in,IDN,TMRC14_APAC_1
2,1379423796357001218,angat,tegakan #HukumHRS jangan biarkan bebas provoka...,in,PHL,TMRC14_APAC_1
3,1382310663410774021,baru,"Rendahkan Martabat peradilan, Rizieq Shihab di...",in,IDN,TMRC14_APAC_1
4,1383295471058259970,baru,Terbukti nyata bhw pemrintah terbuka terhdp ma...,in,IDN,TMRC14_APAC_1
...,...,...,...,...,...,...
198836,1215191677482127360,8a,#เมียหลวงบวกเมียน้อย https://t.co/IWZW7tp8aa,und,PHL,thailand_092020
198837,1227491832889634817,8a,#ผบทบต้องลาออก แล้วเสรีรวมไอซ์ทำไมไม่ออก 555 h...,th,PHL,thailand_092020
198838,1192277843427749888,raya,น้ำตาคนลำพระยา จ.ยะลา\n#Savelampraya #Saveyala...,th,IDN,thailand_092020
198839,1197394755228798976,8a,https://t.co/lomfgaqMyW https://t.co/8aAzjAXcJl,und,PHL,thailand_092020


In [4]:
pd.set_option('display.max_colwidth', None)

has_link = text_relevant_df[text_relevant_df["tweet_text"].str.contains("https://")]
notexpected = has_link[~has_link["project_substring"].isin(["8a", "4b"])]
notexpected[["tweet_text", "project_substring", "tweet_language"]].head(10)

,tweet_text,project_substring,tweet_language
0,Munarman telah diciduk ini lah dukungan dri anak muda endonesia yg gax menyukai aksi terrorisme di endonesia https://t.co/FSVKr4SdKZ,muda,in
2,"tegakan #HukumHRS jangan biarkan bebas provokator akhir zaman itu #FreeIBHRSandUlama. fitnah akhir jaman memang sangat kejam, stop provokasi umat https://t.co/ypjfWK41SU",angat,in
3,"Rendahkan Martabat peradilan, Rizieq Shihab dinilai bsa terancam pidana baru hakim sidang rizieq hrs tegas https://t.co/pY9yMuayHQ",baru,in
4,"Terbukti nyata bhw pemrintah terbuka terhdp masukan n krj ma dr masyrakat, niy niyh nyokap kota baru Indonesia. https://t.co/KGSGp2zwCG",baru,in
5,"Terbukti nyata bhw pemrintah terbuka terhdp masukan n krj ma dr masyrakat, niy niyh nyokap kota baru Indonesia. https://t.co/KGSGp2zwCG",kota,in
6,Mengapa setelah 7 thn Munarman baru ditangkap bcoz munarman telah mberikan smngt utk jihad kpd para plaku Bom sehingga bermunculan pasangan pengantin jihad https://t.co/jajaBwT4ag,baru,in
7,Dukungan serta himbauan Aliansi Mahasiswa dan Aktivis Indonesia dlam penangkapan paska Sdr. Munarman https://t.co/as4EgAaskd,penang,in
8,saatnya rkyt indonesia potong tumpeng merayakan ditangkapnya Munarman. Akhirnya doa qt dikabulkan …. https://t.co/OufIrpihMt,raya,in
10,"aggta Komisi A dari Fraksi PSI, Bro @willsarana mengatakan Pemprov DKI baru mulai bertindak saat ramai diberitakan, padahal kerumunan massa mulai melompat seminggu terakhir di berbagai tempat seperti di pasar, pusat perbelanjaan n tempat wisata. https://t.co/2Tj0vmnrMi",baru,in
13,Mengapa setelah 7 tthn Munarman baru ditangkap coz munarman telah memberikan smngat tuk jihad kepada para pelaku Bom sehingga bermunculan pasangan pengantin jihad https://t.co/sWKY32gc6k,baru,in


In [5]:
## upon inspection of the tweets, many of the incorrectly identified tweets were mainly 2 types
## 1. ones with project_substring in the URL 
## 2. ones with project_substring as only a substring in a word with no relation
## case 2 will be dealt with in tokenization, case 1 will be filtered out here

def substring_in_link_or_mention(tweet, substring):
    words = tweet.split()
    for word in words:
        if "http" in word or word.startswith("@") or word.endswith(":") or "/" in word:
            if substring in word:
                return True
    return False

filtered_link_df = text_relevant_df[~text_relevant_df.apply(lambda row: substring_in_link_or_mention(row["tweet_text"], row["project_substring"]), axis=1)]


In [6]:
has_link = filtered_link_df[filtered_link_df["tweet_text"].str.contains("https://")]
cur = has_link[has_link["project_substring"].isin(["8a", "4b"])]
cur.head(15)


## 4b/8a are not very relevant to the BRI, but further filtering will be dealt with using tokenization

,tweet_id,project_substring,tweet_text,tweet_language,project_country,CampaignID
9223,1503341211335409665,4b,2nd Test: WICKET! Dimuth Karunaratne b Jasprit Bumrah 107 (174b). Sri Lanka 204/7 (56.5 Ovs). #INDvSL #INDvSL https://t.co/w5koTFviTi,in,MLS,TMRC14_APAC_2
11670,1206456438974902277,4b,"As part of its effort to promote local sports and encourage local sportsmen, the #Patiala Brigade of the Fire &amp; Fury Corps organized the Durbuk #IceHockey Challenge at the Durbuk Ice Hockey Rink in eastern #Ladakh.\n#youth4bbi #MondayMotivation #MondayMood https://t.co/sj4pDSYSe3",en,MLS,TMRC14_APAC_2
13069,1391953457100189696,8a,To open from 8am to 12pm till May 17; People seek more relaxations with SOPs in view of Eid.\n#AlAqsaMosque #COVID19 #digitalart #FreePalestine #GazaUnderAttack #Gaza #IsraeliAttackonAlAqsa #Israel #Jerusalem #lockdown #Palestine #PalestinianLivesMatter #SavePalestine #SAFEMOON https://t.co/cM1toXv5dp,en,PHL,TMRC14_APAC_2
14177,1369680500999159808,4b,Pride of Kashmir\nशिक्षा प्रणाली को सुधारें .. देश के भविष्य को सुधारें .. \n#ideas4betterKashmir #Mahashivaratri2021 #NayaKashmir https://t.co/cWG3pAbHMM,hi,MLS,TMRC14_APAC_2
14712,1415155859856904199,8a,@AamirQasim6 Railways says the existing two trains starting from Banihal will be extended to Baramulla from Wednesday whereas another train service will start from Baramulla from 8am..........#greenforkashmir #Kashmir #railway #goodmorning #Pakistan #RedForKashmir https://t.co/sXeSbAfmT2,en,PHL,TMRC14_APAC_2
14830,1390151303758704646,8a,"The daughters of Hurriyat leader Shahid-ul-Islam Wednesday appealed Home Minister Amit Shah to release their ailing father from Delhi’s Tihar Jail.The two daughters of Islam, Suzain Shah,18and Sundas Shah,14,appealed the Home Minister about the release of their father in a letter https://t.co/mY21uaegai",en,PHL,TMRC14_APAC_2
14838,1391965926908518401,8a,"In view of upcoming Eid-ul-Fitr falling this week, the District Administration Srinagar announced partial reopening of bakery, mutton and poultry shops in Srinagar.The order reads, partially permissible activities shall be allowed from 8am to 12pm till May 17. https://t.co/dbpYVMf6WW",en,PHL,TMRC14_APAC_2
15156,1070177134205902849,4b,#JammuAndKashmir recorded an overall voter turnout of 75.3% in the 7th phase of panchayat elections.\n\nPolling took place in as many as 30blocks in 15 districts of the state. These include 14blocks of 6districts in #Jammu region &amp; 16 blocks falling in 9districts of Kashmir Valley. https://t.co/u1OmxzXOm8,en,MLS,TMRC14_APAC_2
17784,1557389854698221569,4b,SoftBank to book $34b gain on Alibaba shares; reduce stake to 14.6% https://t.co/WuMUkCS5Tz,en,MLS,TMRC15_APAC_3
18889,1387659770325204994,8a,People enjoyed watching the liftoff of the Tianhe module this morning (Thu 11:18am 29th April 21 Beijing Time) from the Wenchang Spacecraft Launch Site on Hainan island https://t.co/rhYr4PqnmF,en,PHL,TMRC15_APAC_3


In [7]:
filtered_link_df["tweet_language"].unique()

array(['in', 'en', 'et', 'tl', 'no', 'und', 'hi', 'bn', 'ht', 'de', 'ur',
       'ca', 'es', 'ro', 'pt', 'cy', 'fr', 'sv', 'zh', 'tr', 'ja', 'it',
       'fi', 'nl', nan, 'pl', 'id', 'da', 'is', 'cs', 'eu', 'vi', 'ko',
       'hr', 'lv', 'lt', 'sl', 'hu', 'bs', 'ar', 'th'], dtype=object)

In [8]:

model_dict = {'en': spacy.load('en_core_web_sm'), 'weaker': spacy.load('xx_ent_wiki_sm'), 'better':spacy.load('xx_sent_ud_sm'), 
              'zh': spacy.load('zh_core_web_sm'), 'ja': spacy.load('ja_core_news_sm'), 'es': spacy.load('es_core_news_sm'), 
              'ru': spacy.load('ru_core_news_sm'), 'fr': spacy.load('fr_core_news_sm'), 'ro': spacy.load('ro_core_news_sm'),
              'nl': spacy.load('nl_core_news_sm'), 'pl': spacy.load('pl_core_news_sm'), 'pt': spacy.load('pt_core_news_sm'),
              'fi': spacy.load('fi_core_news_sm'), 'de': spacy.load('de_core_news_sm'), 'ca': spacy.load('ca_core_news_sm'),
              'pt': spacy.load('pt_core_news_sm'), 'ko': spacy.load('ko_core_news_sm'), 'ja': spacy.load('ja_core_news_sm'), 
              'ru': spacy.load('ru_core_news_sm')}
model_dict['better'].add_pipe("language_detector")

In [9]:
def get_ents(row):
    lang = row["tweet_language"]
    text = row["tweet_text"]
    ent_lst = []
    if type(text) != str:
        text = str(text)
    if lang in model_dict:
        NER = model_dict[lang]
    else:
        NER = model_dict['better']
    ner_text = NER(text)
    for word in ner_text.ents:
        tuple = (word.text, word.label_)
        ent_lst.append(tuple)
    return ent_lst

filtered_link_df["ents"] = filtered_link_df.apply(get_ents, axis=1)
filtered_link_df

/var/folders/r4/sgr1zrxx0038xwsxj6n3k8900000gn/T/ipykernel_80478/3864816274.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_link_df["ents"] = filtered_link_df.apply(get_ents, axis=1)


,tweet_id,project_substring,tweet_text,tweet_language,project_country,CampaignID,ents
0,1387933360429748225,muda,Munarman telah diciduk ini lah dukungan dri anak muda endonesia yg gax menyukai aksi terrorisme di endonesia https://t.co/FSVKr4SdKZ,in,MLS,TMRC14_APAC_1,[]
2,1379423796357001218,angat,"tegakan #HukumHRS jangan biarkan bebas provokator akhir zaman itu #FreeIBHRSandUlama. fitnah akhir jaman memang sangat kejam, stop provokasi umat https://t.co/ypjfWK41SU",in,PHL,TMRC14_APAC_1,[]
3,1382310663410774021,baru,"Rendahkan Martabat peradilan, Rizieq Shihab dinilai bsa terancam pidana baru hakim sidang rizieq hrs tegas https://t.co/pY9yMuayHQ",in,IDN,TMRC14_APAC_1,[]
4,1383295471058259970,baru,"Terbukti nyata bhw pemrintah terbuka terhdp masukan n krj ma dr masyrakat, niy niyh nyokap kota baru Indonesia. https://t.co/KGSGp2zwCG",in,IDN,TMRC14_APAC_1,[]
5,1383295471058259970,kota,"Terbukti nyata bhw pemrintah terbuka terhdp masukan n krj ma dr masyrakat, niy niyh nyokap kota baru Indonesia. https://t.co/KGSGp2zwCG",in,MLS,TMRC14_APAC_1,[]
...,...,...,...,...,...,...,...
198803,1205092558906380288,raya,RT @pluchanat: 52 ornately crafted river barges are proceeding along the Chao Phraya River. These are such very stunning pictures! #ขบวนพยุ…,en,IDN,thailand_092020,"[(52, CARDINAL), (the Chao Phraya River, FAC), (ขบวนพยุ, MONEY)]"
198807,1205347689690349569,raya,RT @pluchanat: 52 ornately crafted river barges are proceeding along the Chao Phraya River. These are such very stunning pictures! #ขบวนพยุ…,en,IDN,thailand_092020,"[(52, CARDINAL), (the Chao Phraya River, FAC), (ขบวนพยุ, MONEY)]"
198809,1205087530141437953,raya,RT @pluchanat: 52 ornately crafted river barges are proceeding along the Chao Phraya River. These are such very stunning pictures! #ขบวนพยุ…,en,IDN,thailand_092020,"[(52, CARDINAL), (the Chao Phraya River, FAC), (ขบวนพยุ, MONEY)]"
198823,1203281556527181824,ratu,RT @ger_slagger: #Teamkao #TheFaceMenThailand3 ยินดีด้วย you are the winner Congratulations นุ้งบอส 🎉🎉🎉🎉 https://t.co/a1BkhbfxdL,en,IDN,thailand_092020,"[(#Teamkao, MONEY)]"


In [10]:
filtered_link_df["ents"]

0                                                                       []
2                                                                       []
3                                                                       []
4                                                                       []
5                                                                       []
                                        ...                               
198803    [(52, CARDINAL), (the Chao Phraya River, FAC), (ขบวนพยุ, MONEY)]
198807    [(52, CARDINAL), (the Chao Phraya River, FAC), (ขบวนพยุ, MONEY)]
198809    [(52, CARDINAL), (the Chao Phraya River, FAC), (ขบวนพยุ, MONEY)]
198823                                                 [(#Teamkao, MONEY)]
198838                                                                  []
Name: ents, Length: 171045, dtype: object

In [16]:
empty_df = filtered_link_df[filtered_link_df["ents"].apply(lambda x: not x)]
empty_df.head(20)

,tweet_id,project_substring,tweet_text,tweet_language,project_country,CampaignID,ents
0,1387933360429748225,muda,Munarman telah diciduk ini lah dukungan dri anak muda endonesia yg gax menyukai aksi terrorisme di endonesia https://t.co/FSVKr4SdKZ,in,MLS,TMRC14_APAC_1,[]
2,1379423796357001218,angat,"tegakan #HukumHRS jangan biarkan bebas provokator akhir zaman itu #FreeIBHRSandUlama. fitnah akhir jaman memang sangat kejam, stop provokasi umat https://t.co/ypjfWK41SU",in,PHL,TMRC14_APAC_1,[]
3,1382310663410774021,baru,"Rendahkan Martabat peradilan, Rizieq Shihab dinilai bsa terancam pidana baru hakim sidang rizieq hrs tegas https://t.co/pY9yMuayHQ",in,IDN,TMRC14_APAC_1,[]
4,1383295471058259970,baru,"Terbukti nyata bhw pemrintah terbuka terhdp masukan n krj ma dr masyrakat, niy niyh nyokap kota baru Indonesia. https://t.co/KGSGp2zwCG",in,IDN,TMRC14_APAC_1,[]
5,1383295471058259970,kota,"Terbukti nyata bhw pemrintah terbuka terhdp masukan n krj ma dr masyrakat, niy niyh nyokap kota baru Indonesia. https://t.co/KGSGp2zwCG",in,MLS,TMRC14_APAC_1,[]
6,1388692255339282437,baru,Mengapa setelah 7 thn Munarman baru ditangkap bcoz munarman telah mberikan smngt utk jihad kpd para plaku Bom sehingga bermunculan pasangan pengantin jihad https://t.co/jajaBwT4ag,in,IDN,TMRC14_APAC_1,[]
7,1387927204563148800,penang,Dukungan serta himbauan Aliansi Mahasiswa dan Aktivis Indonesia dlam penangkapan paska Sdr. Munarman https://t.co/as4EgAaskd,in,MLS,TMRC14_APAC_1,[]
8,1387043613876514825,raya,saatnya rkyt indonesia potong tumpeng merayakan ditangkapnya Munarman. Akhirnya doa qt dikabulkan …. https://t.co/OufIrpihMt,in,IDN,TMRC14_APAC_1,[]
10,1389199997237821447,baru,"aggta Komisi A dari Fraksi PSI, Bro @willsarana mengatakan Pemprov DKI baru mulai bertindak saat ramai diberitakan, padahal kerumunan massa mulai melompat seminggu terakhir di berbagai tempat seperti di pasar, pusat perbelanjaan n tempat wisata. https://t.co/2Tj0vmnrMi",in,IDN,TMRC14_APAC_1,[]
11,1378984595123658753,baru,"RT @bintangemon: Aku yang baru olahraga sekali setelah 2 bulan:\n\n“Gila sehat banget gua, tuanya pasti kayak sandiaga uno nih”",in,IDN,TMRC14_APAC_1,[]


In [13]:
nonempty_df = filtered_link_df[filtered_link_df["ents"].notna() & ~filtered_link_df["ents"].apply(lambda x: not x)]
nonempty_df.head(20)

,tweet_id,project_substring,tweet_text,tweet_language,project_country,CampaignID,ents
91,1453257563928244224,baru,"RT @cxarfgyvhu: @deeznuts_junior yah padahal baru mutualan:( be happy!! don't forget to smile, fight and take care of your health &lt;333",en,IDN,TMRC14_APAC_1,"[(RT @cxarfgyvhu, PERSON), (health &lt;333, ORG)]"
1348,1454652409330876422,angat,"be happy always! Galau mulu hobi lu. Tapi gapapa no matter what remember u always have me olrait? And always remember that u are very really sangat precious, biutipul. And worth it! I lop u bestie &lt;3 jamed ew,. shombonk gk bsa di tag",en,PHL,TMRC14_APAC_1,"[(Galau, GPE), (bestie &, ORG)]"
1467,1422388159992647689,bali,"jujur mirissss bgt sama kaya gini, having sex is a choice, but sometime people just take a decision without concerning about the risk and the right way. Ya balik lagi, ga semuanya punya privilege buat dapet education ttg itu juga",en,IDN,TMRC14_APAC_1,"[(jujur mirissss bgt, ORG), (kaya gini, PERSON), (ga, ORG), (semuanya punya, ORG), (juga, ORG)]"
1492,1469662365226467332,awar,"You are educated, but your thoughts and language not matched to your personality. Dusaro pr kichad uchhalne se phale khud ko dekho miss ritika. Ap ek anpadh gawaro ki tarah agar tipadiya krti firogi to education ka matlb kya rah jayega. So don't spread negativity.",en,IDN,TMRC14_APAC_1,"[(khud ko dekho, PERSON), (ki tarah agar tipadiya krti firogi, PERSON), (rah jayega, PERSON)]"
1767,1337299165588512768,ppp,"RT @leesavmark: ppp mutualan kuy , all fandom sabi esp nctzen wayzennie , rt + like ya thankies !",en,IDN,TMRC14_APAC_1,"[(RT, GPE)]"
1768,1337299165588512768,ppp,"RT @leesavmark: ppp mutualan kuy , all fandom sabi esp nctzen wayzennie , rt + like ya thankies !",en,MLS,TMRC14_APAC_1,"[(RT, GPE)]"
2225,1359006339813498881,baru,RT @whojhyun: unflop please🥺\nmau cari mutual baru nctzen wayzenni yuk mutualan. help rt like yaa thankyou😊,en,IDN,TMRC14_APAC_1,"[(🥺\nmau cari mutual baru, PERSON), (yuk mutualan, PERSON), (help rt, PERSON), (yaa thankyou, PERSON)]"
2336,1387279873865965572,ncr,RT @skiesforsuk: mathyung and maknae are always incredible 💜🔥💜🔥\n#CHOIHYUNSUK #최현석 #チェ・ヒョンソク #崔玹硕 \n@treasuremembers https://t.co/9kIzJg5yhJ,en,PHL,TMRC14_APAC_1,"[(RT @skiesforsuk, ORG), (#CHOIHYUNSUK #, MONEY), (#, CARDINAL), (崔玹硕 \n, PERSON)]"
3373,1351021243265392644,bali,RT @lucisthetic: indonesian twitter during the bule bali conflict: https://t.co/XPqP4OLTNz,en,IDN,TMRC14_APAC_1,"[(indonesian, NORP)]"
5768,1266738080326828032,ratu,Great job Jordan and Sarah and congratulations!,en,IDN,TMRC14_APAC_1,"[(Jordan, PERSON), (Sarah, PERSON)]"


In [19]:
print(f"size of filtered_df: {len(filtered_link_df)}, size of nonempty_df: {len(nonempty_df)}, size of empty_df: {len(empty_df)}")
print(f"languages of nonempty_df: {nonempty_df['tweet_language'].unique()}")
print(f"languages of empty_df: {empty_df['tweet_language'].unique()}")

size of filtered_df: 171045, size of nonempty_df: 17575, size of empty_df: 153470
languages of nonempty_df: ['en' 'de' 'ca' 'es' 'pt' 'ro' 'fr' 'zh' 'ja' 'fi' 'nl' 'pl' 'ko']
languages of empty_df: ['in' 'et' 'en' 'tl' 'no' 'und' 'hi' 'bn' 'ht' 'ur' 'ro' 'cy' 'sv' 'tr'
 'it' 'zh' nan 'pl' 'id' 'fi' 'pt' 'da' 'is' 'cs' 'eu' 'es' 'vi' 'hr' 'lv'
 'lt' 'ca' 'sl' 'fr' 'hu' 'bs' 'ar' 'nl' 'de' 'ja' 'ko' 'th']


In [ ]:
filtered_link_df.to_csv("~/Coding/buntain/sentiment_data/filtered_entitied_df.csv", encoding='utf-8', quoting=csv.QUOTE_ALL)
